<a href="https://colab.research.google.com/github/womaro/SimlationOfQubit/blob/main/ToBeCorrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classical Cost of Transmitting a Qubit: Protocol Visualization and Simulation**

**Introduction to the Research**

The simulation code implements the groundbreaking protocol described in the paper "Classical Cost of Transmitting a Qubit" by Renner, Tavakoli, and Quintino. This research addresses a fundamental question in quantum information theory: what is the minimum number of classical bits needed to perfectly simulate the transmission of a quantum bit (qubit)?
The authors prove a remarkable result—exactly two classical bits are both necessary and sufficient to simulate all possible correlations that could be obtained by transmitting a qubit, even when the receiver can perform the most general quantum measurements possible (POVMs). This result extends the previous work by Toner and Bacon, which only covered the more restricted case of projective measurements.

**The Classical Simulation Protocol**

ny quantum state to Bob using only two classical bits, along with pre-shared randomness. Here's how it works:

1. Shared Randomness Setup: Alice and Bob share two normalized random vectors λ₁ and λ₂ uniformly distributed on the unit sphere in 3D space (represented by the Bloch sphere).
2. Classical Communication: Instead of sending a quantum state ρ (represented as a point on the Bloch sphere), Alice calculates and sends two bits:

c₁ = H(ρ·λ₁) and c₂ = H(ρ·λ₂)
where H is the Heaviside function (H(z) = 1 if z ≥ 0, and H(z) = 0 if z < 0)


3. Vector Transformation: Bob transforms the shared vectors based on the received bits:

If c₁ = 0, Bob flips λ₁ → -λ₁; otherwise, he leaves it unchanged
If c₂ = 0, Bob flips λ₂ → -λ₂; otherwise, he leaves it unchanged


4. Measurement Simulation: When Bob needs to perform a measurement corresponding to some POVM elements, he uses these transformed vectors to compute the correct probabilities for each outcome.

The visualization in this simulation shows exactly how the Heaviside function creates a hemispherical partitioning of the Bloch sphere, and how the transformation of vectors enables Bob to reproduce quantum statistics without actually receiving a quantum state.

**Mathematical Insight**

The mathematical beauty of this protocol lies in how it leverages the geometry of the Bloch sphere. The Heaviside function H(ρ·λ) effectively determines whether the vector λ lies in the same hemisphere as the quantum state ρ. The two bits sent by Alice provide Bob with enough information to correctly orient the random vectors relative to the original quantum state.
The protocol exploits the fact that once these vectors are properly oriented, the statistics of any measurement on the original quantum state can be reproduced using only classical calculations involving these vectors. This reveals a profound connection between quantum information and classical communication complexity.

**SIC-POVM Extension**

As an extension to the basic protocol, the code also implements Symmetric Informationally Complete POVMs (SIC-POVMs). These are optimal measurement sets that form a regular tetrahedron in the Bloch sphere representation. The SIC-POVM visualization demonstrates how these special measurement sets can be rotated while preserving their critical mathematical properties.
This extension connects to the paper's discussion of general POVMs and shows how the classical simulation protocol applies even to these sophisticated measurement structures that go beyond standard projective measurements.

**Visualization Components**

The simulation provides three key visualizations:

1. Distribution Analysis: Verifies the statistical properties of the randomly generated points on the Bloch sphere, confirming they follow the expected uniform distribution.
2. Protocol Visualization: Shows the quantum state, the shared random vectors, and their transformed versions on the Bloch sphere, with the critical hemisphere where H(ρ·λ) = 1 highlighted.
3. SIC-POVM Visualization: Demonstrates how canonical SIC-POVM sets transform under rotation while maintaining their mathematical properties.

Together, these visualizations provide an intuitive geometric understanding of the theoretical concepts in the research paper, making abstract mathematical proofs tangible and accessible.

**Significance and Applications**

This research and implementation have profound implications for understanding the boundary between quantum and classical information. By proving that exactly two bits are both necessary and sufficient for simulating qubit communication, the paper establishes a precise quantification of the quantum-classical gap in this scenario.

The protocol implemented here could potentially be used in quantum network simulations, educational tools for quantum information theory, and as a building block for more complex quantum protocols that rely on classical communication channels.

**Imports and Utility Functions**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import random
from scipy.spatial import distance
from scipy import stats

# ==================== UTILITY FUNCTIONS ====================
def heaviside(z):
    """Implementation of Heaviside function H(z)."""
    return 1 if z >= 0 else 0

def cartesian_to_spherical(point):
    """Converts a point from Cartesian (x,y,z) to spherical (theta, phi) coordinates."""
    x, y, z = point
    r = np.sqrt(x**2 + y**2 + z**2)  # Should be 1 for unit sphere

    theta = np.arccos(z / r) * 180 / np.pi

    phi = np.arctan2(y, x) * 180 / np.pi
    if phi < 0:
        phi += 360  # Convert to 0-360 range

    return theta, phi

def bloch_to_ket(point):
    """Converts a point on the Bloch sphere to quantum state in ket notation."""
    x, y, z = point

    # Calculate theta and phi
    theta = np.arccos(z)
    phi = np.arctan2(y, x)

    # Calculate coefficients
    alpha = np.cos(theta/2)
    beta_mag = np.sin(theta/2)
    beta_phase = phi

    # Format complex numbers nicely
    if abs(alpha) < 1e-10:
        alpha_str = "0"
    elif abs(1 - alpha) < 1e-10:
        alpha_str = "1"
    elif abs(-1 - alpha) < 1e-10:
        alpha_str = "-1"
    else:
        alpha_str = f"{alpha:.4f}"

    if abs(beta_mag) < 1e-10:
        beta_str = "0"
    else:
        # Handle the phase factor
        if abs(beta_phase) < 1e-10:
            beta_str = f"{beta_mag:.4f}"
        elif abs(abs(beta_phase) - np.pi) < 1e-10:
            # Phase is π or -π
            beta_str = f"-{beta_mag:.4f}"
        else:
            # Express as a complex number
            re = beta_mag * np.cos(beta_phase)
            im = beta_mag * np.sin(beta_phase)
            if abs(re) < 1e-10:
                re_str = ""
            elif abs(re) == 1:
                re_str = "-" if re < 0 else ""
            else:
                re_str = f"{re:.4f}"

            if abs(im) < 1e-10:
                beta_str = re_str or "0"
            elif abs(im - 1) < 1e-10 and not re_str:
                beta_str = "i"
            elif abs(im + 1) < 1e-10 and not re_str:
                beta_str = "-i"
            elif abs(im) == 1:
                beta_str = f"{re_str}{'-' if im < 0 else '+'}i"
            else:
                beta_str = f"{re_str}{'-' if im < 0 else '+'}{abs(im):.4f}i"

    if beta_str.startswith("+"):
        beta_str = beta_str[1:]

    # Create the ket notation - using ASCII characters for compatibility
    if abs(beta_mag) < 1e-10:
        return "|0>"
    elif abs(alpha) < 1e-10:
        return "|1>"
    else:
        return f"{alpha_str}|0> + {beta_str}|1>"

def get_bloch_vector_from_pure_state(psi):
    """Calculates the Bloch vector corresponding to a pure quantum state."""
    # Pauli matrices
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_y = np.array([[0, -1j], [1j, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])

    # Calculate the expected values of Pauli matrices
    x = np.real(psi.conj().T @ sigma_x @ psi)[0, 0]
    y = np.real(psi.conj().T @ sigma_y @ psi)[0, 0]
    z = np.real(psi.conj().T @ sigma_z @ psi)[0, 0]

    return np.array([x, y, z])

def get_pure_state_from_bloch_vector(r):
    """Calculates the pure quantum state corresponding to a Bloch vector."""
    # Normalize the Bloch vector (should have length 1)
    r = r / np.linalg.norm(r)

    # Convert to spherical coordinates
    theta = np.arccos(r[2])  # Zenithal angle (from z-axis)
    phi = np.arctan2(r[1], r[0])  # Azimuthal angle (in xy-plane)

    # Quantum state in vector representation
    psi = np.array([[np.cos(theta/2)],
                   [np.sin(theta/2) * np.exp(1j * phi)]])

    return psi

def get_density_matrix_from_pure_state(psi):
    """Calculates the density matrix corresponding to a pure quantum state."""
    # For a pure state |psi⟩, the density matrix is |psi⟩⟨psi|
    return psi @ psi.conj().T

# Global dictionary to store simulation data across cells
simulation_data = {}

**Protocol Functions**

In [ ]:
def generate_random_sphere_points(n):
    """Generates n points randomly distributed on a unit sphere."""
    print(f"Generating {n} points randomly on a unit sphere...")

    start_time = time.time()

    # Truly random method using gaussian distribution to avoid pole bias
    points = np.random.randn(n, 3)
    # Normalize to unit sphere
    norms = np.sqrt(np.sum(points**2, axis=1))
    points = points / norms[:, np.newaxis]

    end_time = time.time()
    print(f"Generation completed in {end_time - start_time:.2f} seconds.")

    return points

def generate_random_quantum_state():
    """Generates a random quantum state represented as a point on the Bloch sphere."""
    # Generate random vector in 3D space
    rho = np.random.randn(3)
    # Normalize to unit sphere
    rho = rho / np.linalg.norm(rho)

    print(f"Generated random quantum state ρ = [{rho[0]:.6f}, {rho[1]:.6f}, {rho[2]:.6f}]")

    # Convert to spherical coordinates and quantum notation
    theta, phi = cartesian_to_spherical(rho)
    ket_notation = bloch_to_ket(rho)

    print(f"Spherical coordinates: θ = {theta:.4f}°, φ = {phi:.4f}°")
    print(f"State in ket notation: {ket_notation}")

    return rho

def select_random_points(points, num_select=2):
    """Randomly selects a specified number of points from the given array."""
    indices = np.random.choice(len(points), num_select, replace=False)
    return points[indices]

def compute_transformed_vectors(selected_points, quantum_state):
    """Computes transformed vectors lambda' based on quantum state and Heaviside function."""
    transformed_points = []

    for i, point in enumerate(selected_points):
        # Calculate value c_i = H(rho⋅lambda_i)
        c_i = heaviside(np.dot(quantum_state, point))

        # Transform vector lambda_i to lambda_i'
        # lambda_i' = lambda_i when c_i = 1
        # lambda_i' = -lambda_i when c_i = 0
        lambda_prime = point if c_i == 1 else -point

        transformed_points.append({
            'original': point,
            'transformed': lambda_prime,
            'c_i': c_i
        })

    return transformed_points

def calculate_angles_with_rho(points, rho):
    """Calculates angles between each point and the quantum state rho."""
    # Calculate dot products
    dot_products = np.dot(points, rho)

    # Convert to angles (in degrees)
    angles = np.arccos(np.clip(dot_products, -1.0, 1.0)) * 180 / np.pi

    return angles

def generate_and_select_points(n=1000):
    """Main function to generate points, select lambda vectors, and generate quantum state."""
    # Generate quantum state
    quantum_state = generate_random_quantum_state()

    # Generate points
    all_points = generate_random_sphere_points(n)

    # Select random points
    selected_points = select_random_points(all_points)

    # Calculate angles between points and quantum state
    angles = calculate_angles_with_rho(selected_points, quantum_state)

    # Print information about selected points
    print("\nSelected points (lambda1, lambda2):")
    for i, point in enumerate(selected_points):
        theta, phi = cartesian_to_spherical(point)
        print(f"λ{i+1} = [{point[0]:.6f}, {point[1]:.6f}, {point[2]:.6f}]")
        print(f"Spherical coordinates: θ = {theta:.2f}°, φ = {phi:.2f}°")
        print(f"Angle with ρ: {angles[i]:.2f}°")

    # Compute transformed points
    transformed_points = compute_transformed_vectors(selected_points, quantum_state)

    # Print information about transformed points
    print("\nTransformed points (lambda1', lambda2'):")
    for i, point_info in enumerate(transformed_points):
        original = point_info['original']
        transformed = point_info['transformed']
        c_i = point_info['c_i']

        # Calculate angles
        dot_product = np.dot(quantum_state, original)

        print(f"c{i+1} = H(ρ⋅λ{i+1}) = H({dot_product:.4f}) = {c_i}")

        if c_i == 1:
            print(f"λ{i+1}' = λ{i+1} (no change)")
        else:
            print(f"λ{i+1}' = -λ{i+1}")

        print(f"λ{i+1}' = [{transformed[0]:.6f}, {transformed[1]:.6f}, {transformed[2]:.6f}]")

    # Store data for use in other cells
    simulation_data['protocol'] = {
        'all_points': all_points,
        'selected_points': selected_points,
        'quantum_state': quantum_state,
        'transformed_points': transformed_points
    }

    return all_points, selected_points, quantum_state, transformed_points

def generate_protocol_simulation(n_points=1000):
    """Generates inputs for the quantum protocol and simulates the first steps."""
    all_points, selected_points, quantum_state, transformed_points = generate_and_select_points(n_points)

    # Calculate probabilities for POVM elements
    # This is a simple example with a projective measurement
    p1 = 0.7
    p2 = 0.3
    y1 = np.array([0.0, 0.0, 1.0])  # |0>
    y2 = np.array([0.0, 0.0, -1.0]) # |1>

    # Calculate quantum probabilities using Born's rule
    prob1 = p1 * (1 + np.dot(quantum_state, y1))
    prob2 = p2 * (1 + np.dot(quantum_state, y2))

    # Normalize
    total = prob1 + prob2
    prob1 /= total
    prob2 /= total

    # Get quantum state in ket notation
    ket_notation = bloch_to_ket(quantum_state)

    print("\nQuantum simulation protocol:")
    print(f"Quantum state ρ = {quantum_state} → |ψ> = {ket_notation}")

    for i, point_info in enumerate(transformed_points):
        print(f"Bit {i+1}: c{i+1} = {point_info['c_i']}")

    print(f"POVM elements: p₁ = {p1}, y₁ = {y1}, p₂ = {p2}, y₂ = {y2}")
    print(f"Quantum probabilities: P(1) = {prob1:.4f}, P(2) = {prob2:.4f}")

    # Implementation of protocol steps
    print("\nImplementation of protocol steps:")
    print("1. Alice and Bob share two unit vectors λ₁, λ₂ ∈ ℝ³")
    print("2. Instead of sending the qubit ρ, Alice calculates two bits:")
    print(f"   c₁ = H(ρ⋅λ₁) = {transformed_points[0]['c_i']}")
    print(f"   c₂ = H(ρ⋅λ₂) = {transformed_points[1]['c_i']}")
    print("3. Bob reverses each vector λᵢ when the corresponding bit cᵢ is 0:")

    # Format transformed vectors
    for i, point_info in enumerate(transformed_points):
        if point_info['c_i'] == 1:
            print(f"   λ{i+1}' = λ{i+1} (no change, c{i+1}=1)")
        else:
            print(f"   λ{i+1}' = -λ{i+1} (reversed, c{i+1}=0)")

    # Store additional simulation data
    simulation_data['protocol']['povm'] = {
        1: (p1, y1),
        2: (p2, y2)
    }
    simulation_data['protocol']['quantum_probs'] = {
        1: prob1,
        2: prob2
    }

    return simulation_data['protocol']

**SIC-POVM Functions**

In [ ]:
def generate_haar_rotation_matrix():
    """Generates a rotation matrix according to the Haar distribution."""
    # Generate a random 3x3 matrix from normal distribution
    A = np.random.randn(3, 3)

    # Perform QR decomposition of matrix A
    Q, R = np.linalg.qr(A)

    # Correct the sign of the determinant by multiplying by a diagonal matrix
    # containing the signs of diagonal elements of R
    diag = np.diag(np.sign(np.diag(R)))
    Q = Q @ diag

    # Make sure the matrix has determinant 1 (it's a rotation matrix, not a reflection)
    if np.linalg.det(Q) < 0:
        # Reverse the sign of the first column, which changes the sign of the determinant
        Q[:, 0] = -Q[:, 0]

    return Q

def create_canonical_sic_povm():
    """Creates a canonical SIC-POVM set for a qubit."""
    # Bloch vectors forming an ideal tetrahedron inscribed in the unit sphere
    bloch_vectors = [
        np.array([0, 0, 1]),                    # North pole
        np.array([2*np.sqrt(2)/3, 0, -1/3]),    # Remaining three vertices of the tetrahedron
        np.array([-np.sqrt(2)/3, np.sqrt(6)/3, -1/3]),
        np.array([-np.sqrt(2)/3, -np.sqrt(6)/3, -1/3])
    ]

    # Convert Bloch vectors to quantum states
    sic_povm = [get_pure_state_from_bloch_vector(r) for r in bloch_vectors]

    return sic_povm, bloch_vectors

def apply_haar_rotation_to_povm(canonical_povm, canonical_bloch_vectors=None):
    """Applies a random rotation from the Haar distribution to the entire POVM set."""
    # Generate random rotation from Haar distribution
    R = generate_haar_rotation_matrix()

    # If Bloch vectors are not provided, calculate them
    if canonical_bloch_vectors is None:
        canonical_bloch_vectors = [get_bloch_vector_from_pure_state(psi) for psi in canonical_povm]

    # Apply rotation to each Bloch vector
    rotated_bloch_vectors = [R @ r for r in canonical_bloch_vectors]

    # Convert rotated Bloch vectors back to quantum states
    rotated_povm = [get_pure_state_from_bloch_vector(r) for r in rotated_bloch_vectors]

    return rotated_povm, rotated_bloch_vectors, R

def verify_sic_povm(povm_set, bloch_vectors=None, verbose=True):
    """Verifies whether a set of vectors forms a valid SIC-POVM for a qubit."""
    # Expected scalar product for SIC-POVM in dimension d=2
    expected_overlap = 1/3

    # Calculate Bloch vectors if not provided
    if bloch_vectors is None:
        bloch_vectors = [get_bloch_vector_from_pure_state(psi) for psi in povm_set]

    # Check all pairs of vectors
    n = len(povm_set)
    max_error = 0

    # Calculate scalar products of quantum states
    quantum_overlaps = []
    for i in range(n):
        for j in range(i+1, n):
            overlap = np.abs(povm_set[i].conj().T @ povm_set[j])[0, 0] ** 2
            quantum_overlaps.append(overlap)
            error = abs(overlap - expected_overlap)
            max_error = max(max_error, error)

    # Calculate scalar products of Bloch vectors
    bloch_dot_products = []
    for i in range(n):
        for j in range(i+1, n):
            dot_product = np.dot(bloch_vectors[i], bloch_vectors[j])
            bloch_dot_products.append(dot_product)

    # Analyze statistics
    q_overlap_mean = np.mean(quantum_overlaps)
    q_overlap_std = np.std(quantum_overlaps)
    b_dot_mean = np.mean(bloch_dot_products)
    b_dot_std = np.std(bloch_dot_products)

    # Check if scalar products are close enough to the expected value
    is_valid = all(np.isclose(overlap, expected_overlap, rtol=1e-10, atol=1e-10) for overlap in quantum_overlaps)

    if verbose:
        print(f"Quantum scalar products: mean = {q_overlap_mean:.6f}, std dev = {q_overlap_std:.6e}")
        print(f"Bloch scalar products: mean = {b_dot_mean:.6f}, std dev = {b_dot_std:.6e}")
        print(f"Maximum quantum scalar product error: {max_error:.6e}")
        print(f"Is the set a valid SIC-POVM: {is_valid}")

    return is_valid

def generate_sic_povms(num_sets=1):
    """Generates SIC-POVM sets and validates them."""
    # Create canonical SIC-POVM set
    canonical_povm, canonical_bloch_vectors = create_canonical_sic_povm()

    print("Canonical SIC-POVM set:")
    verify_sic_povm(canonical_povm, canonical_bloch_vectors)

    # List to store all valid sets
    valid_povm_sets = []

    for i in range(num_sets):
        print(f"\nGenerating SIC-POVM set #{i+1}:")

        # Apply random rotation from Haar distribution to the canonical set
        rotated_povm, rotated_bloch_vectors, rotation_matrix = apply_haar_rotation_to_povm(
            canonical_povm, canonical_bloch_vectors)

        # Verify the set
        is_valid = verify_sic_povm(rotated_povm, rotated_bloch_vectors)

        if is_valid:
            # Add to the list of valid sets
            valid_povm_sets.append((rotated_povm, rotated_bloch_vectors, rotation_matrix))

    print(f"\nGenerated {len(valid_povm_sets)} valid SIC-POVM sets out of {num_sets} attempts.")

    # Store data for use in other cells
    simulation_data['sic_povm'] = {
        'canonical_bloch_vectors': canonical_bloch_vectors,
        'valid_sets': valid_povm_sets
    }

    if valid_povm_sets:
        simulation_data['sic_povm']['first_valid'] = {
            'rotated_bloch_vectors': valid_povm_sets[0][1],
            'rotation_matrix': valid_povm_sets[0][2],
            'selected_vector_index': random.randint(0, 3)
        }

    return valid_povm_sets

**Visualization Functions**

In [ ]:
def analyze_point_coordinates_distribution(all_points):
    """Analyzes the distribution of x, y, z coordinates of the generated points."""
    if 'protocol' not in simulation_data or 'all_points' not in simulation_data['protocol']:
        print("No points data available. Please run the protocol simulation first.")
        return None

    all_points = simulation_data['protocol']['all_points']

    # Extract x, y, z coordinates
    x_coords = all_points[:, 0]
    y_coords = all_points[:, 1]
    z_coords = all_points[:, 2]

    # Create plot
    fig = make_subplots(rows=1, cols=3, subplot_titles=['X Coordinates', 'Y Coordinates', 'Z Coordinates'])

    # X coordinates histogram
    fig.add_trace(
        go.Histogram(
            x=x_coords,
            nbinsx=30,
            marker_color='red',
            opacity=0.7,
            name='X Coordinates'
        ),
        row=1, col=1
    )

    # Y coordinates histogram
    fig.add_trace(
        go.Histogram(
            x=y_coords,
            nbinsx=30,
            marker_color='green',
            opacity=0.7,
            name='Y Coordinates'
        ),
        row=1, col=2
    )

    # Z coordinates histogram
    fig.add_trace(
        go.Histogram(
            x=z_coords,
            nbinsx=30,
            marker_color='blue',
            opacity=0.7,
            name='Z Coordinates'
        ),
        row=1, col=3
    )

    # Add theoretical distribution line for points on a unit sphere
    x = np.linspace(-1, 1, 100)
    y = np.sqrt(1 - x**2) * (len(all_points)/60)  # Scale for comparison with histogram

    for i in range(1, 4):
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y,
                mode='lines',
                line=dict(color='black', dash='dash', width=2),
                name='Theoretical distribution' if i == 1 else None,
                showlegend=i == 1
            ),
            row=1, col=i
        )

    # Configure axes
    for i in range(1, 4):
        fig.update_xaxes(title_text=f"{'X' if i==1 else 'Y' if i==2 else 'Z'} Coordinate", range=[-1.1, 1.1], row=1, col=i)
        fig.update_yaxes(title_text="Number of points", row=1, col=i)

    # Configure the entire plot
    fig.update_layout(
        title_text='Distribution of x, y, z coordinates of points generated on a unit sphere',
        height=500,
        width=1200,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )

    # Show statistics
    print("Coordinate distribution statistics:")
    print(f"Mean X: {np.mean(x_coords):.4f}, Standard deviation: {np.std(x_coords):.4f}")
    print(f"Mean Y: {np.mean(y_coords):.4f}, Standard deviation: {np.std(y_coords):.4f}")
    print(f"Mean Z: {np.mean(z_coords):.4f}, Standard deviation: {np.std(z_coords):.4f}")

    # Kolmogorov-Smirnov test for comparison with theoretical distribution
    # For uniform distribution on a sphere, coordinates should approximately
    # conform to a specific theoretical distribution (not uniform)
    x_theory = np.random.normal(0, 1/np.sqrt(3), 10000)
    y_theory = np.random.normal(0, 1/np.sqrt(3), 10000)
    z_theory = np.random.normal(0, 1/np.sqrt(3), 10000)

    # Normalize to unit sphere
    norms = np.sqrt(x_theory**2 + y_theory**2 + z_theory**2)
    x_theory /= norms
    y_theory /= norms
    z_theory /= norms

    ks_x = stats.ks_2samp(x_coords, x_theory)
    ks_y = stats.ks_2samp(y_coords, y_theory)
    ks_z = stats.ks_2samp(z_coords, z_theory)

    print("\nKolmogorov-Smirnov test (comparison with theoretical distribution):")
    print(f"X coordinates: statistic={ks_x.statistic:.4f}, p-value={ks_x.pvalue:.4f}")
    print(f"Y coordinates: statistic={ks_y.statistic:.4f}, p-value={ks_y.pvalue:.4f}")
    print(f"Z coordinates: statistic={ks_z.statistic:.4f}, p-value={ks_z.pvalue:.4f}")

    if min(ks_x.pvalue, ks_y.pvalue, ks_z.pvalue) > 0.05:
        print("The distribution of points on the sphere is consistent with the expected uniform distribution (p > 0.05)")
    else:
        print("The distribution of points may deviate from the expected uniform distribution (p ≤ 0.05)")

    return fig

def visualize_protocol_points_plotly():
    """Visualizes points on a unit sphere including the protocol steps using Plotly."""
    if 'protocol' not in simulation_data:
        print("No protocol data available. Please run the protocol simulation first.")
        return None

    all_points = simulation_data['protocol']['all_points']
    selected_points = simulation_data['protocol']['selected_points']
    transformed_points = simulation_data['protocol']['transformed_points']
    quantum_state = simulation_data['protocol']['quantum_state']
    title = "Visualization of Points on Unit Sphere with Protocol Steps"

    # Creating the sphere grid
    phi = np.linspace(0, 2*np.pi, 100)
    theta = np.linspace(0, np.pi, 50)
    phi_grid, theta_grid = np.meshgrid(phi, theta)

    x_sphere = np.sin(theta_grid) * np.cos(phi_grid)
    y_sphere = np.sin(theta_grid) * np.sin(phi_grid)
    z_sphere = np.cos(theta_grid)

    # Creating the plot
    fig = go.Figure()

    # Adding the sphere grid
    fig.add_trace(go.Surface(
        x=x_sphere, y=y_sphere, z=z_sphere,
        colorscale=[[0, 'rgb(240, 240, 240)'], [1, 'rgb(240, 240, 240)']],
        opacity=0.2,
        showscale=False,
        name='Unit Sphere'
    ))

    # Add axes
    axis_length = 1.2  # Extend beyond the sphere

    # X-axis (red)
    fig.add_trace(go.Scatter3d(
        x=[-axis_length, axis_length], y=[0, 0], z=[0, 0],
        mode='lines',
        line=dict(color='red', width=4),
        name='X-axis'
    ))
    fig.add_trace(go.Scatter3d(
        x=[axis_length], y=[0], z=[0],
        mode='text',
        text=['X'],
        textposition="middle right",
        textfont=dict(color='red', size=16),
        showlegend=False
    ))

    # Y-axis (green)
    fig.add_trace(go.Scatter3d(
        x=[0, 0], y=[-axis_length, axis_length], z=[0, 0],
        mode='lines',
        line=dict(color='green', width=4),
        name='Y-axis'
    ))
    fig.add_trace(go.Scatter3d(
        x=[0], y=[axis_length], z=[0],
        mode='text',
        text=['Y'],
        textposition="middle right",
        textfont=dict(color='green', size=16),
        showlegend=False
    ))

    # Z-axis (blue)
    fig.add_trace(go.Scatter3d(
        x=[0, 0], y=[0, 0], z=[-axis_length, axis_length],
        mode='lines',
        line=dict(color='blue', width=4),
        name='Z-axis'
    ))
    fig.add_trace(go.Scatter3d(
        x=[0], y=[0], z=[axis_length],
        mode='text',
        text=['Z'],
        textposition="top center",
        textfont=dict(color='blue', size=16),
        showlegend=False
    ))

    # Sample a subset of points for display if there are too many
    display_points = all_points
    if len(all_points) > 2000:
        indices = np.random.choice(len(all_points), 2000, replace=False)
        display_points = all_points[indices]

    # Adding all points
    fig.add_trace(go.Scatter3d(
        x=display_points[:, 0], y=display_points[:, 1], z=display_points[:, 2],
        mode='markers',
        marker=dict(
            size=4,
            color='blue',
            opacity=0.2
        ),
        name='Random Points'
    ))

    # Adding quantum state (rho)
    theta_rho, phi_rho = cartesian_to_spherical(quantum_state)
    ket_notation = bloch_to_ket(quantum_state)

    hover_text = f"ρ<br>x: {quantum_state[0]:.4f}, y: {quantum_state[1]:.4f}, z: {quantum_state[2]:.4f}<br>" + \
                 f"θ: {theta_rho:.2f}°, φ: {phi_rho:.2f}°<br>{ket_notation}"

    # Add transparent hemisphere representing the space where H(ρ⋅λ) = 1
    theta_hemi = np.linspace(0, np.pi, 50)
    phi_hemi = np.linspace(0, 2*np.pi, 100)
    theta_grid_hemi, phi_grid_hemi = np.meshgrid(theta_hemi, phi_hemi)

    x_hemi = np.sin(theta_grid_hemi) * np.cos(phi_grid_hemi)
    y_hemi = np.sin(theta_grid_hemi) * np.sin(phi_grid_hemi)
    z_hemi = np.cos(theta_grid_hemi)

    # Calculate values on the grid
    hemi_points = np.stack([x_hemi.flatten(), y_hemi.flatten(), z_hemi.flatten()], axis=1)
    dot_products = np.dot(hemi_points, quantum_state)
    mask = dot_products >= 0

    x_hemi_filtered = np.reshape(hemi_points[mask, 0], (-1, 1))
    y_hemi_filtered = np.reshape(hemi_points[mask, 1], (-1, 1))
    z_hemi_filtered = np.reshape(hemi_points[mask, 2], (-1, 1))

    fig.add_trace(go.Scatter3d(
        x=x_hemi_filtered[:, 0], y=y_hemi_filtered[:, 0], z=z_hemi_filtered[:, 0],
        mode='markers',
        marker=dict(
            size=2,
            color='yellow',
            opacity=0.1
        ),
        name='H(ρ⋅λ) = 1 Region'
    ))

    fig.add_trace(go.Scatter3d(
        x=[quantum_state[0]], y=[quantum_state[1]], z=[quantum_state[2]],
        mode='markers+text',
        marker=dict(
            size=10,
            color='yellow',
            symbol='diamond',
            line=dict(
                color='black',
                width=2
            )
        ),
        text=['ρ'],
        hovertext=[hover_text],
        hoverinfo='text',
        textposition="top center",
        name='Quantum State'
    ))

    # Line from center to quantum state
    fig.add_trace(go.Scatter3d(
        x=[0, quantum_state[0]], y=[0, quantum_state[1]], z=[0, quantum_state[2]],
        mode='lines',
        line=dict(
            color='gold',
            width=4,
            dash='solid'
        ),
        showlegend=False
    ))

    # Adding original and transformed points
    colors = ['red', 'green']

    for i, point_info in enumerate(transformed_points):
        original_point = point_info['original']
        transformed_point = point_info['transformed']
        c_i = point_info['c_i']

        theta_orig, phi_orig = cartesian_to_spherical(original_point)
        theta_trans, phi_trans = cartesian_to_spherical(transformed_point)

        hover_orig = f"λ{i+1}<br>x: {original_point[0]:.4f}, y: {original_point[1]:.4f}, z: {original_point[2]:.4f}<br>" + \
                     f"θ: {theta_orig:.2f}°, φ: {phi_orig:.2f}°<br>c{i+1} = H(ρ⋅λ{i+1}) = {c_i}"

        hover_trans = f"λ{i+1}'<br>x: {transformed_point[0]:.4f}, y: {transformed_point[1]:.4f}, z: {transformed_point[2]:.4f}<br>" + \
                      f"θ: {theta_trans:.2f}°, φ: {phi_trans:.2f}°<br>λ{i+1}' = {'' if c_i == 1 else '-'}λ{i+1}"

        # Add original point
        fig.add_trace(go.Scatter3d(
            x=[original_point[0]], y=[original_point[1]], z=[original_point[2]],
            mode='markers+text',
            marker=dict(
                size=8,
                color=colors[i]
            ),
            text=[f'λ{i+1}'],
            hovertext=[hover_orig],
            hoverinfo='text',
            textposition="top center",
            name=f'λ{i+1}'
        ))

        # Add transformed point
        if np.array_equal(original_point, transformed_point):
            # If λ = λ' (no change), mark it with a special marker
            fig.add_trace(go.Scatter3d(
                x=[transformed_point[0]], y=[transformed_point[1]], z=[transformed_point[2]],
                mode='markers+text',
                marker=dict(
                    size=12,
                    color=colors[i],
                    symbol='diamond',
                    line=dict(color='black', width=2)
                ),
                text=['λ = λ\''],
                hovertext=[f"λ{i+1} = λ{i+1}' (no change)"],
                hoverinfo='text',
                textposition="top right",
                name=f'λ{i+1} = λ{i+1}\' (c{i+1}=1)'
            ))
        else:
            # If λ ≠ λ' (there is a change), add a new point
            fig.add_trace(go.Scatter3d(
                x=[transformed_point[0]], y=[transformed_point[1]], z=[transformed_point[2]],
                mode='markers+text',
                marker=dict(
                    size=8,
                    color=colors[i],
                    symbol='circle-open',
                    line=dict(color=colors[i], width=2)
                ),
                text=[f'λ{i+1}\''],
                hovertext=[hover_trans],
                hoverinfo='text',
                textposition="top center",
                name=f'λ{i+1}\' = -λ{i+1} (c{i+1}=0)'
            ))

        # Line from center to original point
        fig.add_trace(go.Scatter3d(
            x=[0, original_point[0]], y=[0, original_point[1]], z=[0, original_point[2]],
            mode='lines',
            line=dict(
                color=colors[i],
                width=3,
                dash='dash'
            ),
            showlegend=False
        ))

        # If there's a change, add a line to the transformed point
        if not np.array_equal(original_point, transformed_point):
            fig.add_trace(go.Scatter3d(
                x=[0, transformed_point[0]], y=[0, transformed_point[1]], z=[0, transformed_point[2]],
                mode='lines',
                line=dict(
                    color=colors[i],
                    width=3,
                    dash='dot'
                ),
                showlegend=False
            ))

    # Layout configuration
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube'
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=14
        )
    )

    return fig

def create_sic_povm_plotly_visualization():
    """Creates an interactive Plotly visualization of a SIC-POVM set on the Bloch sphere."""
    if 'sic_povm' not in simulation_data or 'first_valid' not in simulation_data['sic_povm']:
        print("No valid SIC-POVM data available. Please generate SIC-POVM sets first.")
        return None

    canonical_bloch_vectors = simulation_data['sic_povm']['canonical_bloch_vectors']
    rotated_bloch_vectors = simulation_data['sic_povm']['first_valid']['rotated_bloch_vectors']
    rotation_matrix = simulation_data['sic_povm']['first_valid']['rotation_matrix']
    selected_vector_index = simulation_data['sic_povm']['first_valid']['selected_vector_index']

    # Get the selected vector
    selected_canonical_vector = canonical_bloch_vectors[selected_vector_index]
    selected_rotated_vector = rotated_bloch_vectors[selected_vector_index]

    # Create Plotly figure
    fig = go.Figure()

    # Bloch sphere surface
    phi_grid = np.linspace(0, 2*np.pi, 100)
    theta_grid = np.linspace(0, np.pi, 50)
    phi_mesh, theta_mesh = np.meshgrid(phi_grid, theta_grid)

    x_sphere = np.sin(theta_mesh) * np.cos(phi_mesh)
    y_sphere = np.sin(theta_mesh) * np.sin(phi_mesh)
    z_sphere = np.cos(theta_mesh)

    fig.add_trace(go.Surface(
        x=x_sphere,
        y=y_sphere,
        z=z_sphere,
        colorscale=[[0, 'rgba(240, 240, 240, 0.2)'], [1, 'rgba(240, 240, 240, 0.2)']],
        showscale=False,
        name='Bloch Sphere'
    ))

    # Coordinate axes
    axis_length = 1.2
    axis_colors = ['red', 'green', 'blue']
    axis_labels = ['X', 'Y', 'Z']
    axis_coords = [
        ([-axis_length, axis_length], [0, 0], [0, 0]),
        ([0, 0], [-axis_length, axis_length], [0, 0]),
        ([0, 0], [0, 0], [-axis_length, axis_length])
    ]

    for (x_line, y_line, z_line), color, label in zip(axis_coords, axis_colors, axis_labels):
        fig.add_trace(go.Scatter3d(
            x=x_line, y=y_line, z=z_line,
            mode='lines',
            line=dict(color=color, width=4),
            name=f'{label} Axis'
        ))

    # POVM sets visualization
    povm_sets = [canonical_bloch_vectors, rotated_bloch_vectors]
    povm_colors = ['blue', 'red']
    povm_names = ['Canonical POVM', 'Rotated POVM']

    for povms, color, name in zip(povm_sets, povm_colors, povm_names):
        # Draw tetrahedron edges
        for i in range(4):
            for j in range(i+1, 4):
                fig.add_trace(go.Scatter3d(
                    x=[povms[i][0], povms[j][0]],
                    y=[povms[i][1], povms[j][1]],
                    z=[povms[i][2], povms[j][2]],
                    mode='lines',
                    line=dict(color=color, width=2, dash='dot'),
                    name=f'{name} Edge'
                ))

        # Add POVM vertices
        for k, vec in enumerate(povms):
            # Check if this is the selected vector
            is_selected = (k == selected_vector_index)
            marker_size = 10 if not is_selected else 15
            symbol = 'diamond' if not is_selected else 'diamond-open'
            opacity = 0.7 if not is_selected else 1.0
            line_width = 2 if not is_selected else 3

            fig.add_trace(go.Scatter3d(
                x=[vec[0]], y=[vec[1]], z=[vec[2]],
                mode='markers+text',
                marker=dict(
                    size=marker_size,
                    color=color,
                    symbol=symbol,
                    opacity=opacity,
                    line=dict(color='black', width=line_width)
                ),
                text=[f'{name} {k+1}{"*" if is_selected else ""}'],
                textposition="top center",
                name=f'{name} Vector {k+1}{"*" if is_selected else ""}'
            ))

    # Add transformation arrow for the selected vector
    fig.add_trace(go.Scatter3d(
        x=[selected_canonical_vector[0], selected_rotated_vector[0]],
        y=[selected_canonical_vector[1], selected_rotated_vector[1]],
        z=[selected_canonical_vector[2], selected_rotated_vector[2]],
        mode='lines',
        line=dict(color='purple', width=5, dash='dashdot'),
        name='Selected vector transformation'
    ))

    # Layout configuration
    fig.update_layout(
        title=f'SIC-POVM Rotation Visualization (Selected vector: {selected_vector_index+1})',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube'
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=14
        )
    )

    # Add annotation with information about the selected vector and rotation matrix
    fig.add_annotation(
        x=1.0,
        y=0.0,
        xref="paper",
        yref="paper",
        text=f"<b>Selected vector {selected_vector_index+1}</b><br>" +
             f"Canonical: [{selected_canonical_vector[0]:.4f}, {selected_canonical_vector[1]:.4f}, {selected_canonical_vector[2]:.4f}]<br>" +
             f"Rotated: [{selected_rotated_vector[0]:.4f}, {selected_rotated_vector[1]:.4f}, {selected_rotated_vector[2]:.4f}]<br><br>" +
             f"<b>SO(3) Rotation matrix</b><br>" +
             f"[{rotation_matrix[0,0]:.4f}, {rotation_matrix[0,1]:.4f}, {rotation_matrix[0,2]:.4f}]<br>" +
             f"[{rotation_matrix[1,0]:.4f}, {rotation_matrix[1,1]:.4f}, {rotation_matrix[1,2]:.4f}]<br>" +
             f"[{rotation_matrix[2,0]:.4f}, {rotation_matrix[2,1]:.4f}, {rotation_matrix[2,2]:.4f}]",
        showarrow=False,
        bordercolor="black",
        borderwidth=2,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
    )

    return fig

**SIC-POVM Measurement Function**

In [ ]:
def perform_sic_povm_measurement(quantum_state=None, sic_povm_set=None, num_measurements=1000, visualize=True):
    """
    Performs measurements on a quantum state using SIC-POVM elements and analyzes results.

    Args:
        quantum_state: The quantum state (Bloch vector) to measure (defaults to using the simulation state)
        sic_povm_set: A tuple containing (quantum_states, bloch_vectors, rotation_matrix) (defaults to using first set)
        num_measurements: Number of measurement samples to collect
        visualize: Whether to create visualization of measurement statistics

    Returns:
        dict: Measurement statistics and visualization figure
    """
    print("\n=== Performing SIC-POVM Measurements on Quantum State ===")

    # Get quantum state from simulation data if not provided
    if quantum_state is None:
        if 'protocol' not in simulation_data or 'quantum_state' not in simulation_data['protocol']:
            print("No quantum state available. Please run the protocol simulation first.")
            return None
        quantum_state = simulation_data['protocol']['quantum_state']

    # Get POVM elements from simulation data if not provided
    if sic_povm_set is None:
        if 'sic_povm' not in simulation_data or 'valid_sets' not in simulation_data['sic_povm'] or not simulation_data['sic_povm']['valid_sets']:
            print("No valid SIC-POVM sets available. Please generate SIC-POVM sets first.")
            return None
        sic_povm_set = simulation_data['sic_povm']['valid_sets'][0]

    # Get POVM elements from the set
    rotated_povm, rotated_bloch_vectors, rotation_matrix = sic_povm_set

    # Convert quantum state (if it's a Bloch vector) to density matrix
    if quantum_state.shape == (3,):
        # Quantum state is a Bloch vector
        state_ket = get_pure_state_from_bloch_vector(quantum_state)
        density_matrix = get_density_matrix_from_pure_state(state_ket)
        bloch_vector = quantum_state
    else:
        # Quantum state is already in ket form
        density_matrix = get_density_matrix_from_pure_state(quantum_state)
        bloch_vector = get_bloch_vector_from_pure_state(quantum_state)

    # Calculate measurement probabilities for each POVM element
    # For SIC-POVM with rank-1 projectors Πᵢ = |ψᵢ⟩⟨ψᵢ|/d where d=2 for qubit
    # probability p(i) = Tr(ρΠᵢ) = Tr(ρ|ψᵢ⟩⟨ψᵢ|/d) = ⟨ψᵢ|ρ|ψᵢ⟩/d
    probabilities = []
    d = 2  # dimension (qubit)

    for i, povm_element in enumerate(rotated_povm):
        # Calculate probability: ⟨ψᵢ|ρ|ψᵢ⟩/d
        prob = np.real((povm_element.conj().T @ density_matrix @ povm_element)[0, 0]) / d
        probabilities.append(prob)

    # Normalize probabilities (should sum to 1, but ensure due to numerical precision)
    probabilities = np.array(probabilities)
    probabilities = probabilities / np.sum(probabilities)

    print(f"\nQuantum state: {bloch_to_ket(bloch_vector)}")
    print(f"Measurement probabilities using SIC-POVM:")
    for i, prob in enumerate(probabilities):
        print(f"  POVM element {i+1}: {prob:.6f}")

    # Perform simulated measurements
    measurement_results = np.random.choice(len(rotated_povm), size=num_measurements, p=probabilities)

    # Count occurrences of each outcome
    unique, counts = np.unique(measurement_results, return_counts=True)
    frequencies = counts / num_measurements

    print(f"\nSimulated {num_measurements} measurements:")
    for outcome, freq in zip(unique, frequencies):
        print(f"  Outcome {outcome+1}: {freq:.6f} (theoretical: {probabilities[outcome]:.6f})")

    # Calculate statistical distance between theoretical and experimental distributions
    statistical_distance = 0.5 * np.sum(np.abs(frequencies - probabilities[unique]))
    print(f"\nTotal variation distance between theoretical and experimental: {statistical_distance:.6f}")

    # Create visualization
    fig = None
    fig_bloch = None
    if visualize:
        fig = go.Figure()

        # Bar chart comparing theoretical vs experimental probabilities
        for i in range(len(rotated_povm)):
            # Theoretical probability
            fig.add_trace(go.Bar(
                x=[f'Outcome {i+1}'],
                y=[probabilities[i]],
                name=f'Theoretical {i+1}',
                marker_color='blue',
                opacity=0.6,
                width=0.4,
                offset=-0.2
            ))

            # Experimental frequency (if this outcome was observed)
            if i in unique:
                idx = np.where(unique == i)[0][0]
                fig.add_trace(go.Bar(
                    x=[f'Outcome {i+1}'],
                    y=[frequencies[idx]],
                    name=f'Experimental {i+1}',
                    marker_color='red',
                    opacity=0.6,
                    width=0.4,
                    offset=0.2
                ))
            else:
                # If outcome wasn't observed, show zero frequency
                fig.add_trace(go.Bar(
                    x=[f'Outcome {i+1}'],
                    y=[0],
                    name=f'Experimental {i+1}',
                    marker_color='red',
                    opacity=0.6,
                    width=0.4,
                    offset=0.2
                ))

        # Configure layout
        fig.update_layout(
            title=f'SIC-POVM Measurement Results ({num_measurements} samples)',
            xaxis_title='Measurement Outcome',
            yaxis_title='Probability / Frequency',
            barmode='group',
            bargap=0.15,
            bargroupgap=0.1,
            legend=dict(
                title="Probability Type",
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="center",
                x=0.5
            )
        )

        # Add annotation about statistical distance
        fig.add_annotation(
            x=0.5,
            y=-0.15,
            xref="paper",
            yref="paper",
            text=f"Total variation distance: {statistical_distance:.6f}",
            showarrow=False,
            font=dict(size=14),
            align="center"
        )

        # Also create a 3D visualization showing the quantum state and the POVM elements
        # on the Bloch sphere
        fig_bloch = go.Figure()

        # Add Bloch sphere
        phi_grid = np.linspace(0, 2*np.pi, 50)
        theta_grid = np.linspace(0, np.pi, 25)
        phi_mesh, theta_mesh = np.meshgrid(phi_grid, theta_grid)

        x_sphere = np.sin(theta_mesh) * np.cos(phi_mesh)
        y_sphere = np.sin(theta_mesh) * np.sin(phi_mesh)
        z_sphere = np.cos(theta_mesh)

        fig_bloch.add_trace(go.Surface(
            x=x_sphere, y=y_sphere, z=z_sphere,
            colorscale=[[0, 'rgb(240, 240, 240)'], [1, 'rgb(240, 240, 240)']],
            opacity=0.2,
            showscale=False
        ))

        # Add POVM elements
        for i, bloch_vec in enumerate(rotated_bloch_vectors):
            # Size proportional to probability
            marker_size = 8 + probabilities[i] * 30

            fig_bloch.add_trace(go.Scatter3d(
                x=[bloch_vec[0]],
                y=[bloch_vec[1]],
                z=[bloch_vec[2]],
                mode='markers+text',
                marker=dict(
                    size=marker_size,
                    color=f'rgba({int(255*(1-probabilities[i]))}, 0, {int(255*probabilities[i])}, 0.8)',
                    symbol='circle'
                ),
                text=[f'{i+1}: {probabilities[i]:.4f}'],
                name=f'POVM {i+1} (p={probabilities[i]:.4f})'
            ))

        # Add quantum state
        fig_bloch.add_trace(go.Scatter3d(
            x=[bloch_vector[0]],
            y=[bloch_vector[1]],
            z=[bloch_vector[2]],
            mode='markers+text',
            marker=dict(
                size=12,
                color='gold',
                symbol='diamond'
            ),
            text=['ρ'],
            name='Quantum State ρ'
        ))

        # Add lines from center to each POVM element
        for i, bloch_vec in enumerate(rotated_bloch_vectors):
            fig_bloch.add_trace(go.Scatter3d(
                x=[0, bloch_vec[0]],
                y=[0, bloch_vec[1]],
                z=[0, bloch_vec[2]],
                mode='lines',
                line=dict(
                    color=f'rgba({int(255*(1-probabilities[i]))}, 0, {int(255*probabilities[i])}, 0.5)',
                    width=2
                ),
                showlegend=False
            ))

        # Add line from center to quantum state
        fig_bloch.add_trace(go.Scatter3d(
            x=[0, bloch_vector[0]],
            y=[0, bloch_vector[1]],
            z=[0, bloch_vector[2]],
            mode='lines',
            line=dict(
                color='gold',
                width=3
            ),
            showlegend=False
        ))

        # Configure layout
        fig_bloch.update_layout(
            title='Quantum State and SIC-POVM Elements on Bloch Sphere',
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                aspectmode='cube'
            ),
            margin=dict(l=0, r=0, b=0, t=40)
        )

    # Store the measurement results and figures
    simulation_data['measurement'] = {
        'quantum_state': bloch_vector,
        'probabilities': probabilities,
        'measurement_results': measurement_results,
        'frequencies': {int(unique[i]): frequencies[i] for i in range(len(unique))},
        'statistical_distance': statistical_distance,
        'figure': fig,
        'figure_bloch': fig_bloch
    }

    return simulation_data['measurement']

**Configuration and Main Execution**

In [ ]:
def get_configuration():
    """Get user configuration for the simulation."""
    print("=== Classical Cost of Transmitting a Qubit Protocol Simulation ===")
    print("=============================================================")

    config = {}

    # Number of points on the Bloch sphere
    try:
        n_points = int(input("Enter number of points to generate (default: 1000): ") or "1000")
        if n_points <= 0:
            print("Number of points must be positive. Using default value 1000.")
            n_points = 1000
    except ValueError:
        print("Invalid input. Using default value 1000.")
        n_points = 1000

    config['n_points'] = n_points

    # Number of SIC-POVM sets to generate
    try:
        n_povms = int(input("Enter number of SIC-POVM sets to generate (default: 1): ") or "1")
        if n_povms <= 0:
            print("Number of SIC-POVM sets must be positive. Using default value 1.")
            n_povms = 1
    except ValueError:
        print("Invalid input. Using default value 1.")
        n_povms = 1

    config['n_povms'] = n_povms

    # Number of measurements to simulate
    try:
        n_measurements = int(input("Enter number of measurements to simulate (default: 10000): ") or "10000")
        if n_measurements <= 0:
            print("Number of measurements must be positive. Using default value 10000.")
            n_measurements = 10000
    except ValueError:
        print("Invalid input. Using default value 10000.")
        n_measurements = 10000

    config['n_measurements'] = n_measurements

    # Set random seed for reproducibility
    np.random.seed(42)
    random.seed(42)

    return config

def main():
    """Main execution function"""
    # Get configuration
    config = get_configuration()

    # Run protocol simulation
    print("\n=== Running Protocol Simulation ===")
    protocol_data = generate_protocol_simulation(config['n_points'])

    # Generate SIC-POVM sets
    print("\n=== Generating SIC-POVM Sets ===")
    sic_povm_sets = generate_sic_povms(config['n_povms'])

    # Perform SIC-POVM measurements
    if sic_povm_sets:
        print("\n=== Performing Measurements with SIC-POVM ===")
        measurement_data = perform_sic_povm_measurement(
            num_measurements=config['n_measurements']
        )
    else:
        print("\nNo valid SIC-POVM sets generated. Cannot perform measurements.")

    # Display all visualizations at the end
    print("\n=== Displaying Visualizations ===")

    # First, display protocol visualization
    print("\nShowing protocol simulation visualization...")
    protocol_fig = visualize_protocol_points_plotly()
    if protocol_fig:
        protocol_fig.show()

    # Then, display distribution analysis
    print("\nShowing distribution analysis of points on the unit sphere...")
    distribution_fig = analyze_point_coordinates_distribution(
        simulation_data['protocol']['all_points'] if 'protocol' in simulation_data else None
    )
    if distribution_fig:
        distribution_fig.show()

    # Next, display SIC-POVM visualization
    print("\nShowing SIC-POVM visualization...")
    sic_povm_fig = create_sic_povm_plotly_visualization()
    if sic_povm_fig:
        sic_povm_fig.show()

    # Finally, display measurement visualizations
    if 'measurement' in simulation_data:
        print("\nShowing SIC-POVM measurement results...")
        if simulation_data['measurement']['figure']:
            simulation_data['measurement']['figure'].show()

        print("\nShowing SIC-POVM measurement Bloch sphere visualization...")
        if simulation_data['measurement']['figure_bloch']:
            simulation_data['measurement']['figure_bloch'].show()

    print("\n=== Simulation Completed ===")

    return simulation_data

# Run the main function when this cell is executed
if __name__ == "__main__":
    main()

=== Classical Cost of Transmitting a Qubit Protocol Simulation ===
Enter number of points to generate (default: 1000): 10
Enter number of SIC-POVM sets to generate (default: 1): 10
Enter number of measurements to simulate (default: 10000): 1000

=== Running Protocol Simulation ===
Generated random quantum state ρ = [0.600002, -0.167015, 0.782370]
Spherical coordinates: θ = 38.5219°, φ = 344.4450°
State in ket notation: 0.9440|0> + 0.3178-0.0885i|1>
Generating 10 points randomly on a unit sphere...
Generation completed in 0.00 seconds.

Selected points (lambda1, lambda2):
λ1 = [0.490365, -0.783960, -0.380722]
Spherical coordinates: θ = 112.38°, φ = 302.03°
Angle with ρ: 82.69°
λ2 = [-0.308947, 0.951054, -0.006930]
Spherical coordinates: θ = 90.40°, φ = 108.00°
Angle with ρ: 110.46°

Transformed points (lambda1', lambda2'):
c1 = H(ρ⋅λ1) = H(0.1273) = 1
λ1' = λ1 (no change)
λ1' = [0.490365, -0.783960, -0.380722]
c2 = H(ρ⋅λ2) = H(-0.3496) = 0
λ2' = -λ2
λ2' = [0.308947, -0.951054, 0.006930]


Showing distribution analysis of points on the unit sphere...
Coordinate distribution statistics:
Mean X: 0.1300, Standard deviation: 0.5369
Mean Y: -0.2188, Standard deviation: 0.5663
Mean Z: -0.2977, Standard deviation: 0.4875

Kolmogorov-Smirnov test (comparison with theoretical distribution):
X coordinates: statistic=0.2697, p-value=0.3911
Y coordinates: statistic=0.2770, p-value=0.3594
Z coordinates: statistic=0.3035, p-value=0.2592
The distribution of points on the sphere is consistent with the expected uniform distribution (p > 0.05)



Showing SIC-POVM visualization...



Showing SIC-POVM measurement results...



Showing SIC-POVM measurement Bloch sphere visualization...



=== Simulation Completed ===


This code implements step 4 of the protocol as described in the paper "Classical Cost of Transmitting a Qubit." Here's what it does:

Setup:

Retrieves the transformed λ vectors from protocol steps 1-3
Uses the same SIC-POVM set as in the previous measurement function
Sets up equal probabilities (1/4 each) for selecting POVM elements


Protocol Simulation:

Performs 10,000 experiments to estimate outcome probabilities
In each experiment:

Bob selects a vector y_b according to probability p_b (1/4 for each in SIC-POVM)
Determines which λ' vector to use based on which has larger absolute dot product with y_b
Calculates outcome probabilities using formula (5): p_B(b|{B_b},λ) = p_b θ(y_b·λ) / Σ_j p_j θ(y_j·λ)
Samples a final outcome based on these probabilities




Theoretical Calculations:

Computes the theoretical probabilities based on the protocol
Compares these with:

Experimental results from simulation (empirical frequencies)
Quantum measurement probabilities calculated in the previous step




Visualizations:

Bar chart comparing three sets of probabilities: theoretical protocol, experimental protocol, and quantum
3D Bloch sphere showing the quantum state, transformed λ vectors, and POVM elements


Analysis:

Calculates total variation distance between protocol probabilities and quantum probabilities
Determines whether the protocol successfully simulates quantum statistics



This implementation allows us to verify the key claim of the paper: that two bits of classical communication are sufficient to simulate quantum measurements on a qubit. The expected result should be a very small total variation distance between the protocol's output probabilities and the true quantum measurement probabilities.

In [ ]:
def simulate_bob_protocol_step4():
    """
    Simulates Bob's part of the protocol (step 4) where he uses transformed lambda vectors
    to determine measurement outcome probabilities without directly performing POVM measurements.

    This implements the fourth step of the protocol from "Classical Cost of Transmitting a Qubit":
    Instead of performing a POVM, Bob selects a vector y_b from the POVM elements according to
    probabilities {p_b}, then computes probabilities using eq. (5) from the paper.

    Returns:
        dict: Results and visualizations of the simulation
    """
    # Check if we have all necessary data from previous steps
    if 'protocol' not in simulation_data or 'transformed_points' not in simulation_data['protocol']:
        print("Protocol data not found. Run the protocol simulation first.")
        return None

    if 'sic_povm' not in simulation_data or 'valid_sets' not in simulation_data['sic_povm'] or not simulation_data['sic_povm']['valid_sets']:
        print("No valid SIC-POVM sets available. Generate SIC-POVM sets first.")
        return None

    # Get the quantum state from protocol
    rho_state = simulation_data['protocol']['quantum_state']
    ket_notation = bloch_to_ket(rho_state)

    # Get transformed lambda vectors from protocol
    transformed_points = simulation_data['protocol']['transformed_points']
    lambda_prime1 = transformed_points[0]['transformed']
    lambda_prime2 = transformed_points[1]['transformed']

    # Get the same SIC-POVM set used in the measurement
    povm_set = simulation_data['sic_povm']['valid_sets'][0]
    _, povm_bloch_vectors, _ = povm_set

    # Extract vectors y_b and probabilities p_b from the POVM elements
    # For SIC-POVM, probabilities are 1/4 for each element
    y_vectors = povm_bloch_vectors
    p_values = np.ones(len(y_vectors)) / len(y_vectors)  # Equal probabilities for SIC-POVM

    print("\n=== Simulating Bob's Protocol (Step 4) ===")
    print(f"Quantum state: {ket_notation}")
    print(f"Transformed λ₁': [{lambda_prime1[0]:.4f}, {lambda_prime1[1]:.4f}, {lambda_prime1[2]:.4f}]")
    print(f"Transformed λ₂': [{lambda_prime2[0]:.4f}, {lambda_prime2[1]:.4f}, {lambda_prime2[2]:.4f}]")

    # Calculate theoretical comparison for all possible outcomes
    theoretical_probs = np.zeros(len(y_vectors))
    all_results = []

    # Helper function for Theta function: Θ(z) = z·H(z)
    def theta(z):
        return z if z >= 0 else 0

    # Simulate N experiments
    n_experiments = 10000
    outcome_counts = np.zeros(len(y_vectors))

    for _ in range(n_experiments):
        # Step 1: Bob selects y_b according to probabilities p_b
        selected_index = np.random.choice(len(y_vectors), p=p_values)
        selected_y = y_vectors[selected_index]

        # Step 2: Bob determines which λ' to use
        dot_product1 = abs(np.dot(lambda_prime1, selected_y))
        dot_product2 = abs(np.dot(lambda_prime2, selected_y))

        selected_lambda = lambda_prime1 if dot_product1 >= dot_product2 else lambda_prime2

        # Step 3: Calculate probabilities for all outcomes using formula (5)
        denominators = []
        for j, y_j in enumerate(y_vectors):
            denominators.append(p_values[j] * theta(np.dot(y_j, selected_lambda)))

        denominator_sum = sum(denominators)

        # Only proceed if denominator is not zero
        if denominator_sum > 0:
            outcome_probs = []
            for b, y_b in enumerate(y_vectors):
                # Probability from formula (5)
                prob_b = p_values[b] * theta(np.dot(y_b, selected_lambda)) / denominator_sum
                outcome_probs.append(prob_b)

            # Sample final outcome based on these probabilities
            final_outcome = np.random.choice(len(y_vectors), p=outcome_probs)
            outcome_counts[final_outcome] += 1

            # Store result for this experiment
            all_results.append({
                'selected_y_index': selected_index,
                'selected_lambda': 1 if np.array_equal(selected_lambda, lambda_prime1) else 2,
                'outcome_probs': outcome_probs,
                'final_outcome': final_outcome
            })

    # Calculate experimental probabilities from the outcomes
    experimental_probs = outcome_counts / n_experiments

    # Calculate theoretical probabilities
    # For each possible y_b selection, calculate outcome probabilities
    for i, y_b in enumerate(y_vectors):
        dot_product1 = abs(np.dot(lambda_prime1, y_b))
        dot_product2 = abs(np.dot(lambda_prime2, y_b))

        selected_lambda = lambda_prime1 if dot_product1 >= dot_product2 else lambda_prime2

        denominators = []
        for j, y_j in enumerate(y_vectors):
            denominators.append(p_values[j] * theta(np.dot(y_j, selected_lambda)))

        denominator_sum = sum(denominators)

        # Only add contribution if denominator is not zero
        if denominator_sum > 0:
            # We weight this by the probability p_b of selecting y_b
            weight = p_values[i]
            for b in range(len(y_vectors)):
                theoretical_probs[b] += weight * (p_values[b] * theta(np.dot(y_vectors[b], selected_lambda)) / denominator_sum)

    # Display results
    print("\nTheoretical vs Experimental Probabilities from Protocol Step 4:")
    for b in range(len(y_vectors)):
        print(f"Outcome {b+1}: Theoretical: {theoretical_probs[b]:.6f}, Experimental: {experimental_probs[b]:.6f}")

    # Compare with quantum measurement probabilities if available
    if 'classical_measurement' in simulation_data and 'probabilities' in simulation_data['classical_measurement']:
        quantum_probs = simulation_data['classical_measurement']['probabilities']
        print("\nComparison with Quantum Measurement Probabilities:")
        for b in range(len(y_vectors)):
            print(f"Outcome {b+1}: Protocol: {theoretical_probs[b]:.6f}, Quantum: {quantum_probs[b]:.6f}, " +
                  f"Difference: {abs(theoretical_probs[b]-quantum_probs[b]):.6f}")

        # Calculate total variation distance
        tv_distance = 0.5 * sum(abs(theoretical_probs[b] - quantum_probs[b]) for b in range(len(y_vectors)))
        print(f"\nTotal Variation Distance: {tv_distance:.6f}")

        if tv_distance < 0.01:
            print("The protocol successfully simulates quantum measurement statistics (distance < 0.01)")
        else:
            print("Potential discrepancy between protocol and quantum statistics (distance ≥ 0.01)")

    # Create visualization comparing protocol vs quantum probabilities
    fig = go.Figure()

    # Protocol probabilities
    for i, prob in enumerate(theoretical_probs):
        fig.add_trace(go.Bar(
            x=[f'Outcome {i+1}'],
            y=[prob],
            name=f'Protocol',
            marker_color='blue',
            opacity=0.7,
            width=0.3,
            offset=-0.2
        ))

    # Experimental probabilities from protocol simulation
    for i, prob in enumerate(experimental_probs):
        fig.add_trace(go.Bar(
            x=[f'Outcome {i+1}'],
            y=[prob],
            name=f'Experimental ({n_experiments} runs)',
            marker_color='purple',
            opacity=0.5,
            width=0.3,
            offset=0
        ))

    # Quantum probabilities if available
    if 'classical_measurement' in simulation_data and 'probabilities' in simulation_data['classical_measurement']:
        for i, prob in enumerate(quantum_probs):
            fig.add_trace(go.Bar(
                x=[f'Outcome {i+1}'],
                y=[prob],
                name=f'Quantum',
                marker_color='red',
                opacity=0.7,
                width=0.3,
                offset=0.2
            ))

    # Configure layout
    fig.update_layout(
        title='Protocol vs Quantum Measurement Probabilities',
        xaxis_title='Measurement Outcome',
        yaxis_title='Probability',
        barmode='group',
        annotations=[
            dict(
                x=0.5,
                y=1.15,
                xref='paper',
                yref='paper',
                text=f'State: {ket_notation} | TV Distance: {tv_distance:.6f}',
                showarrow=False,
                font=dict(size=14)
            )
        ]
    )

    # Create 3D visualization
    fig_bloch = go.Figure()

    # Add Bloch sphere
    phi_grid = np.linspace(0, 2*np.pi, 50)
    theta_grid = np.linspace(0, np.pi, 25)
    phi_mesh, theta_mesh = np.meshgrid(phi_grid, theta_grid)

    x_sphere = np.sin(theta_mesh) * np.cos(phi_mesh)
    y_sphere = np.sin(theta_mesh) * np.sin(phi_mesh)
    z_sphere = np.cos(theta_mesh)

    fig_bloch.add_trace(go.Surface(
        x=x_sphere, y=y_sphere, z=z_sphere,
        colorscale=[[0, 'rgb(240, 240, 240)'], [1, 'rgb(240, 240, 240)']],
        opacity=0.2,
        showscale=False
    ))

    # Add quantum state
    fig_bloch.add_trace(go.Scatter3d(
        x=[rho_state[0]],
        y=[rho_state[1]],
        z=[rho_state[2]],
        mode='markers+text',
        marker=dict(
            size=12,
            color='gold',
            symbol='diamond'
        ),
        text=['ρ'],
        name='Quantum State ρ'
    ))

    # Add transformed lambda vectors
    fig_bloch.add_trace(go.Scatter3d(
        x=[lambda_prime1[0]],
        y=[lambda_prime1[1]],
        z=[lambda_prime1[2]],
        mode='markers+text',
        marker=dict(
            size=10,
            color='green',
            symbol='circle'
        ),
        text=['λ₁\''],
        name='λ₁\''
    ))

    fig_bloch.add_trace(go.Scatter3d(
        x=[lambda_prime2[0]],
        y=[lambda_prime2[1]],
        z=[lambda_prime2[2]],
        mode='markers+text',
        marker=dict(
            size=10,
            color='blue',
            symbol='circle'
        ),
        text=['λ₂\''],
        name='λ₂\''
    ))

    # Add POVM vectors y_b with color based on theoretical probabilities
    for i, vec in enumerate(y_vectors):
        fig_bloch.add_trace(go.Scatter3d(
            x=[vec[0]],
            y=[vec[1]],
            z=[vec[2]],
            mode='markers+text',
            marker=dict(
                size=8 + theoretical_probs[i] * 30,
                color=f'rgba({int(255*theoretical_probs[i])}, 0, {int(255*(1-theoretical_probs[i]))}, 0.8)',
                symbol='circle-open'
            ),
            text=[f'y_{i+1}'],
            name=f'POVM y_{i+1} (p={theoretical_probs[i]:.4f})'
        ))

    # Configure layout
    fig_bloch.update_layout(
        title='Protocol Step 4: Quantum State, λ\' Vectors, and POVM Elements',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube'
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    # Store data
    simulation_data['bob_protocol'] = {
        'theoretical_probs': theoretical_probs,
        'experimental_probs': experimental_probs,
        'quantum_probs': quantum_probs if 'classical_measurement' in simulation_data else None,
        'tv_distance': tv_distance if 'classical_measurement' in simulation_data else None,
        'figure_bar': fig,
        'figure_bloch': fig_bloch
    }

    # Display figures
    fig.show()
    fig_bloch.show()

    return simulation_data['bob_protocol']

# Run the function
simulate_bob_protocol_step4()


=== Simulating Bob's Protocol (Step 4) ===
Quantum state: 0.9440|0> + 0.3178-0.0885i|1>
Transformed λ₁': [0.4904, -0.7840, -0.3807]
Transformed λ₂': [0.3089, -0.9511, 0.0069]

Theoretical vs Experimental Probabilities from Protocol Step 4:
Outcome 1: Theoretical: 0.000000, Experimental: 0.000000
Outcome 2: Theoretical: 0.816849, Experimental: 0.818300
Outcome 3: Theoretical: 0.183151, Experimental: 0.181700
Outcome 4: Theoretical: 0.000000, Experimental: 0.000000


UnboundLocalError: cannot access local variable 'tv_distance' where it is not associated with a value

In [ ]:
def simulate_bob_protocol_step4_with_convergence():
    """
    Simulates Bob's part of the protocol (step 4) with a variable number of shots
    and visualizes the convergence of experimental results to theoretical values.

    The function asks the user for a maximum number of shots, then runs the simulation
    iteratively from 1 to max_shots, tracking accuracy at each step.

    Returns:
        dict: Results and visualizations of the simulation
    """
    # Check if we have all necessary data from previous steps
    if 'protocol' not in simulation_data or 'transformed_points' not in simulation_data['protocol']:
        print("Protocol data not found. Run the protocol simulation first.")
        return None

    if 'sic_povm' not in simulation_data or 'valid_sets' not in simulation_data['sic_povm'] or not simulation_data['sic_povm']['valid_sets']:
        print("No valid SIC-POVM sets available. Generate SIC-POVM sets first.")
        return None

    # Get the quantum state from protocol
    rho_state = simulation_data['protocol']['quantum_state']
    ket_notation = bloch_to_ket(rho_state)

    # Get transformed lambda vectors from protocol
    transformed_points = simulation_data['protocol']['transformed_points']
    lambda_prime1 = transformed_points[0]['transformed']
    lambda_prime2 = transformed_points[1]['transformed']

    # Get the same SIC-POVM set used in the measurement
    povm_set = simulation_data['sic_povm']['valid_sets'][0]
    _, povm_bloch_vectors, _ = povm_set

    # Extract vectors y_b and probabilities p_b from the POVM elements
    y_vectors = povm_bloch_vectors
    p_values = np.ones(len(y_vectors)) / len(y_vectors)  # Equal probabilities for SIC-POVM

    # Get quantum measurement probabilities if available
    quantum_probs = None
    if 'classical_measurement' in simulation_data and 'probabilities' in simulation_data['classical_measurement']:
        quantum_probs = simulation_data['classical_measurement']['probabilities']

    # Calculate theoretical protocol probabilities
    theoretical_probs = np.zeros(len(y_vectors))

    # Helper function for Theta function: Θ(z) = z·H(z)
    def theta(z):
        return z if z >= 0 else 0

    # For each possible y_b selection, calculate outcome probabilities
    for i, y_b in enumerate(y_vectors):
        dot_product1 = abs(np.dot(lambda_prime1, y_b))
        dot_product2 = abs(np.dot(lambda_prime2, y_b))

        selected_lambda = lambda_prime1 if dot_product1 >= dot_product2 else lambda_prime2

        denominators = []
        for j, y_j in enumerate(y_vectors):
            denominators.append(p_values[j] * theta(np.dot(y_j, selected_lambda)))

        denominator_sum = sum(denominators)

        # Only add contribution if denominator is not zero
        if denominator_sum > 0:
            # We weight this by the probability p_b of selecting y_b
            weight = p_values[i]
            for b in range(len(y_vectors)):
                theoretical_probs[b] += weight * (p_values[b] * theta(np.dot(y_vectors[b], selected_lambda)) / denominator_sum)

    # Get max shots from user
    try:
        max_shots = int(input("Enter maximum number of shots (default: 10000): ") or "10000")
        if max_shots <= 0:
            print("Number of shots must be positive. Using default value 10000.")
            max_shots = 10000
    except ValueError:
        print("Invalid input. Using default value 10000.")
        max_shots = 10000

    print(f"\n=== Simulating Bob's Protocol (Step 4) with 1 to {max_shots} shots ===")
    print(f"Quantum state: {ket_notation}")

    # Prepare data structures for tracking convergence
    shot_numbers = []
    tv_distances_quantum = []  # Distance to quantum probabilities
    tv_distances_theory = []   # Distance to theoretical protocol probabilities

    # Arrays to store experimental probability values for each outcome
    outcome_probs_over_time = [[] for _ in range(len(y_vectors))]

    # Generate all experimental results upfront
    print("Generating experimental results...")

    # Initialize tracking variables
    all_experiments = []
    cumulative_counts = np.zeros(len(y_vectors))

    # Perform max_shots experiments
    for shot in range(max_shots):
        # Step 1: Bob selects y_b according to probabilities p_b
        selected_index = np.random.choice(len(y_vectors), p=p_values)
        selected_y = y_vectors[selected_index]

        # Step 2: Bob determines which λ' to use
        dot_product1 = abs(np.dot(lambda_prime1, selected_y))
        dot_product2 = abs(np.dot(lambda_prime2, selected_y))

        selected_lambda = lambda_prime1 if dot_product1 >= dot_product2 else lambda_prime2

        # Step 3: Calculate probabilities for all outcomes using formula (5)
        denominators = []
        for j, y_j in enumerate(y_vectors):
            denominators.append(p_values[j] * theta(np.dot(y_j, selected_lambda)))

        denominator_sum = sum(denominators)

        # Only proceed if denominator is not zero
        if denominator_sum > 0:
            outcome_probs = []
            for b, y_b in enumerate(y_vectors):
                # Probability from formula (5)
                prob_b = p_values[b] * theta(np.dot(y_b, selected_lambda)) / denominator_sum
                outcome_probs.append(prob_b)

            # Sample final outcome based on these probabilities
            final_outcome = np.random.choice(len(y_vectors), p=outcome_probs)
            all_experiments.append(final_outcome)

    # Process experiments in increments
    increment = max(1, max_shots // 100)  # Ensure we don't track too many points for very large max_shots

    # For small max_shots, track every shot
    if max_shots <= 1000:
        increments = range(1, max_shots + 1)
    else:
        # For larger max_shots, use logarithmic spacing
        increments = np.logspace(0, np.log10(max_shots), 100).astype(int)
        increments = np.unique(increments)  # Remove duplicates
        increments = np.append(increments, max_shots)  # Ensure we include max_shots

    print(f"Tracking convergence at {len(increments)} points...")

    # Process the experiments
    cumulative_counts = np.zeros(len(y_vectors))

    for n_shots in increments:
        # Update counts with experiments up to n_shots
        for i in range(len(cumulative_counts)):
            cumulative_counts[i] = np.sum(np.array(all_experiments[:n_shots]) == i)

        # Calculate experimental probabilities
        experimental_probs = cumulative_counts / n_shots

        # Store shot number and probabilities
        shot_numbers.append(n_shots)

        # Record probabilities for each outcome
        for i in range(len(y_vectors)):
            outcome_probs_over_time[i].append(experimental_probs[i])

        # Calculate total variation distances
        if quantum_probs is not None:
            tv_distance_quantum = 0.5 * sum(abs(experimental_probs[b] - quantum_probs[b]) for b in range(len(y_vectors)))
            tv_distances_quantum.append(tv_distance_quantum)

        tv_distance_theory = 0.5 * sum(abs(experimental_probs[b] - theoretical_probs[b]) for b in range(len(y_vectors)))
        tv_distances_theory.append(tv_distance_theory)

    # Final experimental probabilities
    final_experimental_probs = cumulative_counts / max_shots

    # Print final results
    print("\nFinal Results after", max_shots, "shots:")
    for b in range(len(y_vectors)):
        print(f"Outcome {b+1}:")
        print(f"  Protocol Theoretical: {theoretical_probs[b]:.6f}")
        print(f"  Protocol Experimental: {final_experimental_probs[b]:.6f}")
        if quantum_probs is not None:
            print(f"  Quantum: {quantum_probs[b]:.6f}")

    if quantum_probs is not None:
        final_tv_quantum = 0.5 * sum(abs(final_experimental_probs[b] - quantum_probs[b]) for b in range(len(y_vectors)))
        print(f"\nFinal Total Variation Distance to Quantum: {final_tv_quantum:.6f}")

    final_tv_theory = 0.5 * sum(abs(final_experimental_probs[b] - theoretical_probs[b]) for b in range(len(y_vectors)))
    print(f"Final Total Variation Distance to Protocol Theory: {final_tv_theory:.6f}")

    # Create convergence plot
    fig_convergence = go.Figure()

    # Add trace for distance to theoretical protocol probabilities
    fig_convergence.add_trace(go.Scatter(
        x=shot_numbers,
        y=tv_distances_theory,
        mode='lines',
        name='Distance to Protocol Theory',
        line=dict(color='blue', width=2)
    ))

    # Add trace for distance to quantum probabilities if available
    if quantum_probs is not None:
        fig_convergence.add_trace(go.Scatter(
            x=shot_numbers,
            y=tv_distances_quantum,
            mode='lines',
            name='Distance to Quantum',
            line=dict(color='red', width=2)
        ))

    # Configure layout
    fig_convergence.update_layout(
        title='Convergence of Protocol Simulation with Increasing Shots',
        xaxis_title='Number of Shots',
        yaxis_title='Total Variation Distance',
        xaxis_type='log',  # Logarithmic x-axis
        yaxis=dict(
            type='log',  # Logarithmic y-axis
            range=[-3, 0]  # From 10^-3 to 10^0
        ),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )

    # Plot individual outcome probabilities over time
    fig_outcomes = go.Figure()

    # Add trace for each outcome
    colors = ['blue', 'red', 'green', 'purple']
    for i in range(len(y_vectors)):
        fig_outcomes.add_trace(go.Scatter(
            x=shot_numbers,
            y=outcome_probs_over_time[i],
            mode='lines',
            name=f'Outcome {i+1}',
            line=dict(color=colors[i % len(colors)], width=2)
        ))

        # Add horizontal lines for theoretical values
        fig_outcomes.add_trace(go.Scatter(
            x=[min(shot_numbers), max(shot_numbers)],
            y=[theoretical_probs[i], theoretical_probs[i]],
            mode='lines',
            line=dict(color=colors[i % len(colors)], width=1, dash='dash'),
            name=f'Theory {i+1}'
        ))

        # Add horizontal lines for quantum values if available
        if quantum_probs is not None:
            fig_outcomes.add_trace(go.Scatter(
                x=[min(shot_numbers), max(shot_numbers)],
                y=[quantum_probs[i], quantum_probs[i]],
                mode='lines',
                line=dict(color=colors[i % len(colors)], width=1, dash='dot'),
                name=f'Quantum {i+1}'
            ))

    # Configure layout
    fig_outcomes.update_layout(
        title='Convergence of Individual Outcome Probabilities',
        xaxis_title='Number of Shots',
        yaxis_title='Probability',
        xaxis_type='log',  # Logarithmic x-axis
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )

    # Create bar chart comparison
    fig_comparison = go.Figure()

    # Protocol theoretical probabilities
    for i, prob in enumerate(theoretical_probs):
        fig_comparison.add_trace(go.Bar(
            x=[f'Outcome {i+1}'],
            y=[prob],
            name='Protocol Theory',
            marker_color='blue',
            opacity=0.7,
            width=0.25,
            offset=-0.25
        ))

    # Protocol experimental probabilities
    for i, prob in enumerate(final_experimental_probs):
        fig_comparison.add_trace(go.Bar(
            x=[f'Outcome {i+1}'],
            y=[prob],
            name=f'Protocol Exp ({max_shots} shots)',
            marker_color='green',
            opacity=0.7,
            width=0.25,
            offset=0
        ))

    # Quantum probabilities if available
    if quantum_probs is not None:
        for i, prob in enumerate(quantum_probs):
            fig_comparison.add_trace(go.Bar(
                x=[f'Outcome {i+1}'],
                y=[prob],
                name='Quantum',
                marker_color='red',
                opacity=0.7,
                width=0.25,
                offset=0.25
            ))

    # Configure layout
    fig_comparison.update_layout(
        title='Comparison of Probabilities',
        xaxis_title='Measurement Outcome',
        yaxis_title='Probability',
        barmode='group',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        )
    )

    # Store data
    simulation_data['bob_protocol_convergence'] = {
        'shot_numbers': shot_numbers,
        'tv_distances_theory': tv_distances_theory,
        'tv_distances_quantum': tv_distances_quantum if quantum_probs is not None else None,
        'theoretical_probs': theoretical_probs,
        'final_experimental_probs': final_experimental_probs,
        'quantum_probs': quantum_probs,
        'outcome_probs_over_time': outcome_probs_over_time,
        'figure_convergence': fig_convergence,
        'figure_outcomes': fig_outcomes,
        'figure_comparison': fig_comparison
    }

    # Display figures
    print("\nDisplaying convergence plots...")
    fig_convergence.show()
    fig_outcomes.show()
    fig_comparison.show()

    return simulation_data['bob_protocol_convergence']

# Run the function
simulate_bob_protocol_step4_with_convergence()

Enter maximum number of shots (default: 10000): 1000

=== Simulating Bob's Protocol (Step 4) with 1 to 1000 shots ===
Quantum state: 0.9440|0> + 0.3178-0.0885i|1>
Generating experimental results...
Tracking convergence at 1000 points...

Final Results after 1000 shots:
Outcome 1:
  Protocol Theoretical: 0.000000
  Protocol Experimental: 0.000000
Outcome 2:
  Protocol Theoretical: 0.816849
  Protocol Experimental: 0.826000
Outcome 3:
  Protocol Theoretical: 0.183151
  Protocol Experimental: 0.174000
Outcome 4:
  Protocol Theoretical: 0.000000
  Protocol Experimental: 0.000000
Final Total Variation Distance to Protocol Theory: 0.009151

Displaying convergence plots...


{'shot_numbers': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,


Proba implementacji Qskit

In [ ]:
# Instalacja wymaganych pakietów
!pip install qiskit matplotlib qiskit.transpile

# Najpierw sprawdźmy wersję zainstalowanego pakietu Qiskit
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

# Import niezbędnych bibliotek
import numpy as np
import matplotlib.pyplot as plt
import math
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit.transpile import PassManager
from qiskit.visualization import plot_histogram
from qiskit.extensions import UnitaryGate

# ===== Funkcje pomocnicze =====
def random_unit_vector(dim=3):
    """Generuje losowy wektor jednostkowy na sferze Blocha"""
    vec = np.random.normal(0, 1, dim)
    return vec / np.linalg.norm(vec)

def bloch_vector_to_state(r):
    """Konwertuje wektor Blocha na stan kwantowy"""
    theta = np.arccos(r[2])
    phi = np.arctan2(r[1], r[0])
    return np.array([np.cos(theta/2), np.exp(1j*phi)*np.sin(theta/2)])

def state_to_bloch_vector(state):
    """Konwertuje stan kwantowy na wektor Blocha"""
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_y = np.array([[0, -1j], [1j, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])

    if len(state.shape) == 1:  # Stan czysty
        rho = np.outer(state, np.conj(state))
    else:  # Już macierz gęstości
        rho = state

    x = np.real(np.trace(rho @ sigma_x))
    y = np.real(np.trace(rho @ sigma_y))
    z = np.real(np.trace(rho @ sigma_z))

    return np.array([x, y, z])

def random_statevector(dim):
    """Generuje losowy wektor stanu kwantowego"""
    state = np.random.normal(0, 1, dim) + 1j * np.random.normal(0, 1, dim)
    state = state / np.linalg.norm(state)
    return state

# ===== POVM Implementations =====
def cross_povm():
    """Cross-POVM z 4 elementami: |0><0|, |1><1|, |+><+|, |-><-|"""
    povm_elements = []
    povm_elements.append(np.array([[1, 0], [0, 0]]) * 0.5)
    povm_elements.append(np.array([[0, 0], [0, 1]]) * 0.5)

    plus_state = np.array([1, 1]) / np.sqrt(2)
    povm_elements.append(np.outer(plus_state, plus_state.conj()) * 0.5)

    minus_state = np.array([1, -1]) / np.sqrt(2)
    povm_elements.append(np.outer(minus_state, minus_state.conj()) * 0.5)

    return povm_elements

# ===== Obliczanie prawdopodobieństwa kwantowego =====
def quantum_probability(state, povm_element):
    """Obliczanie prawdopodobieństwa kwantowego za pomocą reguły Borna: p(k|ρ,{Bₖ}) = tr(ρBₖ)"""
    if len(state.shape) == 1:  # Stan czysty
        return np.real(state.conj() @ povm_element @ state)
    else:  # Macierz gęstości
        return np.real(np.trace(state @ povm_element))

# ===== Implementacja protokołu klasycznego =====
def classical_protocol_pam(rho_bloch, povm_elements):
    """
    Klasyczny protokół przygotuj-i-zmierz używający 2 bitów komunikacji
    """
    # 1. Generowanie dwóch współdzielonych losowych wektorów jednostkowych
    lambda1 = random_unit_vector()
    lambda2 = random_unit_vector()

    # 2. Obliczenie bitów wysyłanych od Alicji do Boba
    c1 = 1 if np.dot(rho_bloch, lambda1) >= 0 else 0
    c2 = 1 if np.dot(rho_bloch, lambda2) >= 0 else 0

    # 3. Bob odwraca wektory, gdy odpowiedni bit wynosi 0
    lambda1_prime = lambda1 if c1 == 1 else -lambda1
    lambda2_prime = lambda2 if c2 == 1 else -lambda2

    # 4. Konwersja elementów POVM na wektory Blocha
    povm_bloch_vectors = []
    for element in povm_elements:
        bloch = state_to_bloch_vector(element)
        povm_bloch_vectors.append(bloch)

    # 5. Bob wybiera wektor z najwyższym iloczynem skalarnym
    selected_lambda = lambda1_prime
    if abs(np.dot(povm_bloch_vectors[0], lambda2_prime)) > abs(np.dot(povm_bloch_vectors[0], lambda1_prime)):
        selected_lambda = lambda2_prime

    # 6. Obliczenie prawdopodobieństw na podstawie iloczynów skalarnych z wybranym lambda
    probabilities = []
    denominator = 0
    numerators = []

    for y_k in povm_bloch_vectors:
        numerator = max(0, np.dot(y_k, selected_lambda))
        numerators.append(numerator)
        denominator += numerator

    if denominator > 0:
        probabilities = [num / denominator for num in numerators]
    else:
        probabilities = [1/len(povm_bloch_vectors)] * len(povm_bloch_vectors)

    return probabilities

# ===== Symulacja kwantowa =====
def quantum_simulation(state, povm_elements, shots=10000):
    """
    Symulacja pomiaru kwantowego za pomocą Qiskit
    """
    # Inicjalizacja Sampler, który zastępuje execute/run w nowszym API
    sampler = Sampler()

    # Przygotowanie obwodu dla Cross-POVM
    qc = QuantumCircuit(2, 2)

    # Przygotowanie stanu
    if len(state.shape) == 1 and len(state) == 2:
        theta = 2 * np.arccos(np.abs(state[0]))
        phi = np.angle(state[1]) - np.angle(state[0])
        qc.u(theta, phi, 0, 0)

    # Uproszczona unitarna dla rozszerzenia Naimarka
    unitary_matrix = np.array([
        [1, 0, 0, 0],
        [0, 1/np.sqrt(2), 1/np.sqrt(2), 0],
        [0, 1/np.sqrt(2), -1/np.sqrt(2), 0],
        [0, 0, 0, 1]
    ])

    unitary_gate = UnitaryGate(unitary_matrix)
    qc.append(unitary_gate, [0, 1])

    # Pomiar
    qc.measure([0, 1], [0, 1])

    # Wykonanie symulacji z użyciem nowego API
    job = sampler.run(qc, shots=shots)
    result = job.result()
    counts = result.quasi_dists[0]

    # Konwersja wyników (nowa struktura wyników w Qiskit 1.0)
    counts_dict = {}
    for bitstring, count in counts.items():
        bits = format(bitstring, f'0{qc.num_clbits}b')
        counts_dict[bits] = count * shots

    # Mapowanie na wyniki POVM - dla Cross-POVM
    povm_probabilities = [0] * len(povm_elements)
    if len(povm_elements) == 4:
        mapping = {'00': 0, '01': 1, '10': 2, '11': 3}
        total_shots = sum(counts_dict.values())
        for outcome, count in counts_dict.items():
            if outcome in mapping:
                povm_probabilities[mapping[outcome]] = count / total_shots

    return povm_probabilities

# ===== Bell Scenario Implementation =====
def bell_singlet_state():
    """Tworzy stan singletowy Bella (|01⟩ - |10⟩)/√2"""
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    qc.x(0)
    qc.z(0)
    return qc

def classical_protocol_bell(measurement_settings, num_iterations=10000):
    """
    Klasyczny protokół scenariusza Bella używający 1 bitu komunikacji
    """
    results = {}

    # Inicjalizacja liczników
    for A_x, B_y in measurement_settings:
        results[f"({A_x},{B_y})"] = {(1, 1): 0, (1, -1): 0, (-1, 1): 0, (-1, -1): 0}

    for _ in range(num_iterations):
        # Współdzielona losowość
        lambda1 = random_unit_vector()
        lambda2 = random_unit_vector()

        for A_x, B_y in measurement_settings:
            # Wektor pomiarowy Alicji
            if A_x == 'X':
                a_vec = np.array([1, 0, 0])
            elif A_x == 'Y':
                a_vec = np.array([0, 1, 0])
            else:  # 'Z'
                a_vec = np.array([0, 0, 1])

            # Wyjście Alicji i bit komunikacyjny
            a = -1 if np.dot(a_vec, lambda1) >= 0 else 1
            c = (1 if np.dot(a_vec, lambda1) >= 0 else -1) * (1 if np.dot(a_vec, lambda2) >= 0 else -1)

            # Bob odwraca lambda2 jeśli c = -1
            lambda2_prime = lambda2 if c == 1 else -lambda2

            # Wektor pomiarowy Boba
            if B_y == 'X':
                b_vec = np.array([1, 0, 0])
            elif B_y == 'Y':
                b_vec = np.array([0, 1, 0])
            else:  # 'Z'
                b_vec = np.array([0, 0, 1])

            # Wyjście Boba
            b = 1 if np.dot(b_vec, lambda2_prime) >= 0 else -1

            # Zapisanie wyniku
            results[f"({A_x},{B_y})"][(a, b)] += 1

    # Konwersja na prawdopodobieństwa i obliczenie korelacji
    for setting, outcomes in results.items():
        if setting.startswith("("):
            total = sum(outcomes.values())
            for outcome in outcomes:
                outcomes[outcome] /= total

            # Obliczenie wartości oczekiwanej
            E_AB = sum(a*b*p for (a, b), p in outcomes.items())
            A_x, B_y = setting[1:-1].split(',')
            results[f"E[{A_x},{B_y}]"] = E_AB

    # Obliczenie wartości CHSH
    if len(measurement_settings) >= 4:
        S = (results["E[X,Z]"] + results["E[X,X]"] +
             results["E[Z,Z]"] - results["E[Z,X]"])
        results["CHSH"] = S

    return results

def bell_scenario_quantum(measurement_settings):
    """Symulacja pomiarów kwantowych w scenariuszu Bella"""
    sampler = Sampler()
    shots = 10000

    # Utworzenie stanu singletowego Bella
    bell_circuit = bell_singlet_state()

    results = {}

    for A_x, B_y in measurement_settings:
        qc = QuantumCircuit(2, 2)
        qc.compose(bell_circuit, inplace=True)

        # Baza pomiarowa Alicji
        if A_x == 'X':
            qc.h(0)
        elif A_x == 'Y':
            qc.sdg(0)
            qc.h(0)

        # Baza pomiarowa Boba
        if B_y == 'X':
            qc.h(1)
        elif B_y == 'Y':
            qc.sdg(1)
            qc.h(1)

        qc.measure([0, 1], [0, 1])

        # Symulacja z użyciem nowego API
        job = sampler.run(qc, shots=shots)
        result = job.result()
        counts = result.quasi_dists[0]

        # Konwersja wyników (nowa struktura wyników w Qiskit 1.0)
        counts_dict = {}
        for bitstring, count in counts.items():
            bits = format(bitstring, f'0{qc.num_clbits}b')
            counts_dict[bits] = count * shots

        # Przetwarzanie wyników
        joint_probs = {}
        for outcome, count in counts_dict.items():
            if len(outcome) == 2:
                b, a = outcome  # Kolejność bitów jest odwrócona w Qiskit
                a_val = -1 if a == '1' else 1
                b_val = -1 if b == '1' else 1
                joint_probs[(a_val, b_val)] = count / shots

        results[f"({A_x},{B_y})"] = joint_probs

        # Obliczenie korelacji
        if joint_probs:
            E_AB = sum(a*b*p for (a, b), p in joint_probs.items())
            results[f"E[{A_x},{B_y}]"] = E_AB

    # Wartość CHSH
    if all(f"E[{A_x},{B_y}]" in results for A_x, B_y in measurement_settings):
        try:
            S = (results["E[X,Z]"] + results["E[X,X]"] +
                 results["E[Z,Z]"] - results["E[Z,X]"])
            results["CHSH"] = S
        except KeyError as e:
            print(f"Brakujący klucz przy obliczaniu CHSH: {e}")

    return results

# ===== Funkcja główna =====
def main():
    """Uruchomienie symulacji"""
    print("============ Scenariusz Przygotuj-i-Zmierz ============")

    # Generowanie losowego stanu kwantowego
    psi = random_statevector(2)
    rho = np.outer(psi, np.conj(psi))
    bloch_vec = state_to_bloch_vector(psi)

    print(f"Losowy stan: {psi}")
    print(f"Wektor Blocha: {bloch_vec}")

    # Użycie Cross-POVM do demonstracji
    povm = cross_povm()

    # Obliczenie teoretycznych prawdopodobieństw kwantowych
    quantum_probs = [quantum_probability(rho, element) for element in povm]
    print("\nTeoretyczne prawdopodobieństwa kwantowe:")
    for i, p in enumerate(quantum_probs):
        print(f"  P({i}) = {p:.4f}")

    # Uruchomienie protokołu klasycznego
    num_trials = 10000
    classical_results = []
    for _ in range(num_trials):
        classical_results.append(classical_protocol_pam(bloch_vec, povm))

    classical_avg = np.mean(classical_results, axis=0)
    print("\nPrawdopodobieństwa protokołu klasycznego (średnia z 10000 przebiegów):")
    for i, p in enumerate(classical_avg):
        print(f"  P({i}) = {p:.4f}")

    # Uruchomienie symulacji kwantowej
    try:
        quantum_sim = quantum_simulation(psi, povm)
        print("\nPrawdopodobieństwa symulacji kwantowej:")
        for i, p in enumerate(quantum_sim):
            print(f"  P({i}) = {p:.4f}")

        # Obliczenie dywergencji KL
        def kl_divergence(p, q):
            epsilon = 1e-10
            p = np.array(p) + epsilon
            q = np.array(q) + epsilon
            return np.sum(p * np.log(p / q))

        kl_theory_classic = kl_divergence(quantum_probs, classical_avg)
        kl_theory_sim = kl_divergence(quantum_probs, quantum_sim)

        print(f"\nDywergencja KL (Teoria vs Klasyczny): {kl_theory_classic:.6f}")
        print(f"Dywergencja KL (Teoria vs Symulacja): {kl_theory_sim:.6f}")

        # Przedstawienie wyników
        plt.figure(figsize=(10, 6))
        x = np.arange(len(quantum_probs))
        width = 0.25

        plt.bar(x - width, quantum_probs, width, label='Teoretyczne')
        plt.bar(x, classical_avg, width, label='Protokół klasyczny')
        plt.bar(x + width, quantum_sim, width, label='Symulacja kwantowa')

        plt.xlabel('Element POVM')
        plt.ylabel('Prawdopodobieństwo')
        plt.title('Scenariusz Przygotuj-i-Zmierz: Protokół klasyczny 2-bitowy vs Kwantowy')
        plt.xticks(x)
        plt.legend()
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Błąd podczas symulacji kwantowej: {e}")
        print("Kontynuacja z samym protokołem klasycznym...")

    print("\n============ Scenariusz Bella ============")

    # Definicja ustawień pomiarowych dla testu CHSH
    measurement_settings = [
        ('X', 'Z'),  # (A₀, B₀)
        ('X', 'X'),  # (A₀, B₁)
        ('Z', 'Z'),  # (A₁, B₀)
        ('Z', 'X')   # (A₁, B₁)
    ]

    try:
        # Uruchomienie symulacji kwantowej Bella
        quantum_bell = bell_scenario_quantum(measurement_settings)

        # Uruchomienie klasycznego protokołu Bella
        classical_bell = classical_protocol_bell(measurement_settings)

        # Wyświetlenie wartości CHSH
        if "CHSH" in quantum_bell and "CHSH" in classical_bell:
            q_chsh = quantum_bell["CHSH"]
            c_chsh = classical_bell["CHSH"]

            print(f"Kwantowa wartość CHSH: {q_chsh:.4f} (Teoretyczna: {2*np.sqrt(2):.4f})")
            print(f"Wartość CHSH protokołu klasycznego: {c_chsh:.4f} (Teoretyczne max: 2)")

            # Przedstawienie korelacji scenariusza Bella
            plt.figure(figsize=(12, 6))
            settings = [f"{A_x},{B_y}" for A_x, B_y in measurement_settings]

            q_corr = [quantum_bell.get(f"E[{s}]", 0) for s in settings]
            c_corr = [classical_bell.get(f"E[{s}]", 0) for s in settings]

            x = np.arange(len(settings))
            width = 0.35

            plt.bar(x - width/2, q_corr, width, label='Kwantowy')
            plt.bar(x + width/2, c_corr, width, label='Protokół klasyczny')

            plt.axhline(y=-0.7071, color='r', linestyle='--', label='Granica CHSH')
            plt.axhline(y=0.7071, color='r', linestyle='--')

            plt.xlabel('Ustawienia pomiarów')
            plt.ylabel('Korelacja E[A,B]')
            plt.title('Scenariusz Bella: Protokół klasyczny 1-bitowy vs Kwantowy')
            plt.xticks(x, settings)
            plt.legend()
            plt.tight_layout()
            plt.show()
        else:
            print("Nie można obliczyć wartości CHSH. Sprawdź wyniki symulacji.")

    except Exception as e:
        print(f"Błąd podczas symulacji scenariusza Bella: {e}")

if __name__ == "__main__":
    main()